In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import os
import sys
sys.path.insert(1, '../../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
_xform_map_ = '''
__id__              = '$[*]._id'              | MovieID      | uniq
__director__        = '$[*].director.name_id' | DirectorID   | uniq
__castmember__      = '$[*].cast.[*].name_id' | CastMemberID | uniq
__id__              --- "hasTitle"       --- '$[*].name'          | xsd:string          ^^^ "imdb_600k_international_movies"
__id__              --- "yearReleased"   --- '$[*].year'          | xsd:date     | yyyy ^^^ "imdb_600k_international_movies"
__id__              --- "runTime"        --- '$[*].runtime'       | xsd:duration | dura ^^^ "imdb_600k_international_movies"
__id__              --- "hasGenre"       --- '$[*].genre[*]'      | xsd.string   | cata ^^^ "imdb_600k_international_movies"
__id__              --- "ratingValue"    --- '$[*].ratingValue'   | xsd:float    | valu ^^^ "imdb_600k_international_movies"
__id__              --- "summary"        --- '$[*].summary_text'  | xsd:string   | cont ^^^ "imdb_600k_international_movies"
__director__        --- "directedMovie"  --- __id__                                     ^^^ "imdb_600k_international_movies"
__director__        --- "hasName"        --- '$[*].director.name' | xsd:string          ^^^ "imdb_600k_international_movies"
__castmember__      --- "castMemberOf"   --- __id__                                     ^^^ "imdb_600k_international_movies"
__castmember__      --- "hasName"        --- '$[*].cast.[*].name' | xsd:string          ^^^ "imdb_600k_international_movies"
'''
ofv = rt.ontologyFrameworkInstance(xform_spec=_xform_map_)
_base_ = '../../../data/kaggle_imdb_600k/international-movies-json/'
_files_ = os.listdir(_base_)
print(f'{len(_files_)} files...')
for i in range(len(_files_)):
    _file_ = _files_[i]
    _txt_  = open(_base_ + _file_).read()
    _json_ = json.loads(_txt_)
    ofv.parse(_json_)
    if i > 2: break

In [ ]:
# rt.chordDiagram(ofv.df_triples.to_pandas(), [('sbj','obj')])
nx_g = rt.createNetworkXGraph(ofv.df_triples, [('sbj','obj')])
pos  = nx.spring_layout(nx_g)

In [ ]:
rt.linkNode(ofv.df_triples, [('sbj','obj')], pos, node_size='small', w=1536, h=1024)